Mijn aanpak is als volgt:


openenen en de eerste lijn kan direct naar een lijst geschreven worden met seeds

Input file opnieuw openen en er doorheen itereren vanaf de tweede lijn.

dan als een empty lijn gevonden wordt op basis van de volgende lijn de naam van een nieuwe dict aanmaken. ook linken van de vorige mapnaam

Dan elke lijn lezen en met split(' ') openen in 3 waarden. dan een for loop maken op de hoeveelheid lijnen (3e value) en op basis daarvan steeds een nieuwe append doen op de dict met key en value. In deze loop elke keer +1

Dus 2 loops, outer loop is de lijnen en inner loop is per lijn een aantal regels in de dict maken op basis van de hoeveelheid lijnen uit de derde value

dan iteraten totdat er een empty lijn is en dan uit de volgende lijn de dictnaam aanmaken en dan de loop weer maken en daaraan toewijzen.

In [94]:
seedstemp = []
seeds = []
headertoken = False
dictionary_master = {} # dictionary of dictionaries whereby 
link_dict = {}
transcat = []
next_cat = {}
with open ('2023_day5_input.txt') as f:
    for index, line in enumerate(f):
        if index == 0:
            seedstemp = line.strip().split(' ')
            seedstemp.pop(0)
            seeds = [int(i) for i in seedstemp]
        elif headertoken == True:
            dictname = line.strip().replace(' ','_')
            dictionary_master[dictname] = {}
            transcat.append(dictname)
            splitname = dictname.split('-to-')
            source_cat = splitname[0]
            splitname2 = splitname[1].split('_')
            next_cat[source_cat] = splitname2[0]
            link_dict[source_cat] = dictname
            headertoken = False
        elif line.strip() == '':
            headertoken = True
        
        #else:
        #    partsofline = line.strip().split(' ')
        #    dest_start, source_start, range_lenght = map(int,partsofline)
        #    dest_start += -1
        #    source_start += -1
        #    for i in range(range_lenght):
        #        dest_start += 1
        #        source_start += 1
        #        dictionary_master[dictname][int(source_start)] = int(dest_start)
#print(f'seeds: ',seeds)
#print(dictionary_master)
#print(link_dict)
#print(transcat)
#print(next_cat)
#input("Press Enter to continue...")
#translate seednumber to following numbers

#create a dict for each seednumber whereby the key are the different dict names and the vale is the value looked up in the dict list, effectively giving for each seednumber the value for each category
#we only now need to solve to process the application of the translation to get to the different numbers in the right order. For that we use the link_dict
seedtrans_dict_master = {}

for seednumber in seeds:
    seedtrans_dict_master[seednumber] = {}
    #print(f'seednumber',seednumber)
    #print(f'seednumber for seednumber in seeds:',seednumber)
    sourcevalue = seednumber
    #print(f'sourcevalue after taking seednumber value',sourcevalue)
    #print(f'sourcevalue',sourcevalue)
    category = 'seed'
    #here we need to create a loop for the lenght of the dict and then change the lookup value based on the link_dict
    for i in range(len(dictionary_master)):
        translationtype = link_dict[category] #this fetches the translation_map to use given the specific category (seeds, soil, etc.)
        #print(f'translatiuon type:',translationtype)
        #print(f'current category',category)
        category = next_cat[category] #this looks up and sets which is the next category to use after a specific category is used for teh next loop iteration
        #print(f'next category', category)
        #print(dictionary_master[translationtype])
        #print(f'source value',sourcevalue)
        dest_value = dictionary_master[translationtype].get(sourcevalue, sourcevalue)  #this fetches the translated value (using the source value) from the translationmap and maps it to destination value. in case no value found dest_value is the same as source value
        #print(f'dest_value',dest_value)
        seedtrans_dict_master[seednumber][category] = dest_value #this adds a line in the dictionary of that specific seednumber. giving the category as key with corresponding dest_value
        #print(f'seedtrans_dict_master_seednumber_category_add', seedtrans_dict_master)
        sourcevalue = dest_value #sets the destination value as the source value for the next iteration of the loop
        #print(f' new source value',source_value)
        #print(f'new source value',source_value)
print(seedtrans_dict_master)
#print(index)
#print (dictname)
#print(dictionary_master)
#print(f'transcat:',transcat)        
#print (seeds)
#for dict in seedtrans_dict_master

lowest_location = float('inf')  # Initialize with a very high number

for attributes in seedtrans_dict_master.values():
    location = attributes.get('location')  # Get the location value
    if location < lowest_location:
        lowest_location = location

print(f'lowest location number',lowest_location)

KeyboardInterrupt: 

Dit was niet zo slim. De echte file input had veel te grote ranges, dus vooraf dictionaries aanmaken is kut. Dus opnieuw  doen en dan kijken of de value in de range is en dan + verschil doen. als niet in range dan originele waarde

In [14]:
#deze code van reddit gehaald van de oplossingen en uitgeplozen hoe dit in elkaar steekt. Nieuw voor mij:
# 1: de * bij een split om de eerste waarde aan seeds en de rest aan mappings toe te voegen
# 2: [1:] na split gebruiken om alles behalve de eerste waarde te gebruiken
# 3: map variable die een function toepast op alle waarden. dit doet hij zodra de toegewezen variable aangeroepen wordt en is daarna doorgelopen en niet meer te gebruiken
# 4: reduce die een function met twee variablen toepast om door een iterable door te lopen. elke iteratie gaat het resultaat van de functie (bv een return) als input naar de volgende iteratie

from functools import reduce

seeds, *mappings = open('2023_day5_input.txt').read().split('\n\n') #with the * it unpacks the first part (split is empty line as it has two times \n\n) into seeds and the rest of the parts as a list of strings
seeds = map(int, seeds.split()[1:]) #This line splits the seeds string by spaces and converts each element (except the first one) to an integer. The result is a map object that needs to be iterated over to get the list of integers.

def lookup(start, mapping): #this defines a function with two variables (as a function with two variables is what can be applied with reduce). note that this mapping is only used in the function and basically takes every mapping sections from the mappings once the reduce function iterates over the mappings
    for m in mapping.split('\n')[1:]: # with [1:] not including the first line which is the title of each list in the mapping. for m is iterating over each line in the mapping as the variable is split by linebreak \n
        dst, src, len = map(int, m.split()) # unpacking the destination start, source start and lenght of the range from the current line
        delta = start - src #by deducting the src from the start (input value which is initially the seed number) we can use the range(len) to see if the delta fits.  another way would have been to use range(src,src+len)
        if delta in range(len): #determines if delta (or the start value) is within the range of items on this line
            return dst + delta #if it is within the range then with this code it translates the value to the dst (destination) value. With return statement it exits the loop and the lookup function
    else: return start # if there is nothing in this mapping in range it returns the same value as the start value and exits the function

print(min(reduce(lookup, mappings, int(s)) for s in seeds)) # here we apply for every s in seeds (by calling seeds it applies the map function) the lookup function for each mapping. This is done by the reduce function. the start value is the value s in seeds every time the lookup function is applied as this is defined as the optional initializer value. the value that comes out of the first iteration is the input for the next iteration. Finally it selects the min value of all the returned values and prints it

226172555


The code you've provided is a Python script that processes a series of mappings and applies these mappings to a list of seed values, ultimately finding the minimum final value after all mappings are applied. Let's break it down step by step:

### Code Explanation

1. **Import and File Reading:**

    ```python
    from functools import reduce

    seeds, *mappings = open('data.txt').read().split('\n\n')
    seeds = map(int, seeds.split()[1:])
    ```

    - `from functools import reduce`: This imports the `reduce` function from the `functools` module. `reduce` is used to apply a function cumulatively to the items of an iterable.
    - `seeds, *mappings = open('data.txt').read().split('\n\n')`: This line reads the contents of 'data.txt', splits it by double newlines (`\n\n`), and unpacks the result into `seeds` and `mappings`. `seeds` gets the first section, and `mappings` gets the rest as a list of sections.
    - `seeds = map(int, seeds.split()[1:])`: This line processes the `seeds` string. It splits the string into a list, skips the first element (which is the word "seeds:"), and then converts the remaining elements to integers.

2. **The `lookup` Function:**

    ```python
    def lookup(start, mapping):
        for m in mapping.split('\n')[1:]:
            dst, src, len = map(int, m.split())
            delta = start - src
            if delta in range(len):
                return dst + delta
        else: return start
    ```

    - This function takes a `start` value and a `mapping` string. The mapping string is split into lines, and each line is further split into three integers: `dst` (destination), `src` (source), and `len` (length).
    - It calculates `delta` as the difference between `start` and `src`.
    - If `delta` is within the range specified by `len`, it returns `dst + delta`. Otherwise, it continues to the next line of the mapping.
    - If no match is found in the entire mapping, it returns the `start` value.

3. **Processing and Finding the Minimum Value:**

    ```python
    print(min(reduce(lookup, mappings, int(s)) for s in seeds))
    ```

    - This line is a bit complex as it combines list comprehension, `reduce`, and `min`.
    - For each seed value `s` in `seeds`, it uses `reduce` to apply the `lookup` function cumulatively across all mappings. The `reduce` function starts with `int(s)` as the initial value and then applies each mapping in order, modifying the value each time based on the mapping rules.
    - After all mappings are applied to a seed, the final value is obtained. This process is repeated for each seed.
    - Finally, `min` is used to find the smallest of these final values.

### With Your Provided Data

- The script reads the mappings and applies each of them in sequence to each seed value. 
- Each mapping potentially changes the value based on its rules. 
- After all mappings are applied, the smallest resulting value is printed.

This script is a sophisticated way to process a series of transformation rules on a set of initial values, demonstrating the power of functions like `map`, `reduce`, and list comprehensions in Python.

The `reduce` function in Python, found in the `functools` module, is a fundamental tool for performing cumulative computations on an iterable. It applies a function of two arguments cumulatively to the items of an iterable, from left to right, so as to reduce the iterable to a single value. 

### How `reduce` Works:

1. **Function Argument:** The function you pass to `reduce` must accept two arguments. On each iteration, `reduce` applies this function to the current cumulative result and the next item in the sequence.

2. **Cumulative Process:** `reduce` starts by taking the first two items from the iterable and applying the function to them. Then it takes the result of that computation and applies the function to it and the next item in the sequence, and so on, until there are no more items left.

3. **Final Result:** The end result is a single value.

### Syntax:

```python
from functools import reduce

def my_function(accumulator, item):
    # process and return a new accumulated value
    return new_accumulated_value

result = reduce(my_function, my_iterable, initializer)
```

- `my_function`: The function to apply. It must accept two parameters: an accumulator (the cumulative result of the reduction so far) and an item (the next item in the sequence).
- `my_iterable`: The iterable to reduce.
- `initializer`: An optional value that is used as the first accumulator. If provided, `reduce` starts with this as the initial value and then proceeds through the iterable. If not provided, `reduce` starts with the first two items of the iterable.

### Example:

```python
from functools import reduce

# A simple function that adds two numbers
def add(x, y):
    return x + y

# A list of numbers
numbers = [1, 2, 3, 4]

# Using reduce to compute the sum of the list
result = reduce(add, numbers)  # result is 10 (1+2+3+4)
```

In this example, `reduce` applies the `add` function first to the first two elements of `numbers` (1 and 2), then to the result (3) and the next element (3), and finally to the last result (6) and the last element (4). The final result is 10.

### Usage Note:

`reduce` is a powerful function, but it's often less readable than other Python constructs like list comprehensions or loops. It's most useful when you need to apply a function to an iterable to produce a single, cumulative result. For more straightforward cases, like summing a list of numbers, Python's built-in functions (like `sum`) or a for-loop might be more readable and appropriate.

Certainly! Let's break down the steps of how the `reduce` function works in the example you provided, where we're using it to sum the numbers in a list `[1, 2, 3, 4]`:

The `reduce` function is called with the `add` function and the list `[1, 2, 3, 4]`. Here's how it processes the list:

1. **First Step:**
   - `x = 1`, `y = 2` (first two elements of the list)
   - `add(1, 2)` is called, which returns `3`.
   - The cumulative result is now `3`.

2. **Second Step:**
   - The cumulative result `3` is used as the new `x`.
   - `y = 3` (the next element in the list).
   - `add(3, 3)` is called, which returns `6`.
   - The cumulative result is now `6`.

3. **Third Step:**
   - The cumulative result `6` is used as the new `x`.
   - `y = 4` (the next element in the list).
   - `add(6, 4)` is called, which returns `10`.
   - The cumulative result is now `10`.

There are no more elements in the list, so `reduce` finishes, and `10` is returned as the final result.

To summarize, the `add` function is applied in this order:

- `add(1, 2)` → returns `3`
- `add(3, 3)` → returns `6`
- `add(6, 4)` → returns `10`

The final result of `reduce(add, [1, 2, 3, 4])` is `10`.

The `*` operator in Python, when used in this context, is known as the unpacking operator. It is used to unpack the elements from iterables (like lists or tuples) into function arguments or into other variables. 

In the line `seeds, *mappings = open('data.txt').read().split('\n\n')`, this operator is used to unpack the contents of the file read and split by double newlines (`\n\n`) into two variables: `seeds` and `mappings`.

Here's a detailed breakdown:

1. `open('data.txt').read().split('\n\n')` reads the entire file, converts it into a single string, and then splits this string into a list of substrings wherever there are double newlines.

2. The unpacking operation with `*` is then used in the assignment:
   
   - `seeds` is assigned the first element of this list (which, based on your data, contains the seed values).
   - `*mappings` is assigned the rest of the elements in the list. The `*` operator here collects all remaining elements in the list and assigns them to `mappings`. As a result, `mappings` becomes a list containing each of the remaining substrings (each representing a mapping).

Example:

Assume the result of `open('data.txt').read().split('\n\n')` is `["seeds: 79 14 55 13", "seed-to-soil map:\n50 98 2\n52 50 48", "soil-to-fertilizer map:\n..."]`.

- `seeds` will be `"seeds: 79 14 55 13"`.
- `mappings` will be a list of the remaining strings: `["seed-to-soil map:\n50 98 2\n52 50 48", "soil-to-fertilizer map:\n...", ...]`.

This use of `*` for unpacking is a very convenient way to split a list into a head (first element) and tail (all other elements) in Python, and it is commonly used for this kind of operation.

If you leave out the `*` in the assignment `seeds, *mappings = open('data.txt').read().split('\n\n')`, the behavior of the assignment operation changes significantly. 

Without the `*`, Python will expect the right side of the assignment (the result of `open('data.txt').read().split('\n\n')`) to have exactly two elements, because you are trying to unpack these elements into two variables (`seeds` and `mappings`). Here's what would happen:

1. **Two Elements Expected:** Python will expect that the split operation results in exactly two elements. If there are more or fewer than two elements, you will get a `ValueError` indicating that there's a mismatch in the number of values to unpack.

2. **Direct Assignment:** The first element of the list will be assigned to `seeds`, and the second element will be assigned to `mappings`. Unlike the original case with `*mappings`, where `mappings` would collect all remaining elements of the list, in this case, `mappings` will just be a single element (the second element from the split list).

For example, with the data you've provided:

- Original data: `["seeds: 79 14 55 13", "seed-to-soil map:\n50 98 2\n52 50 48", "soil-to-fertilizer map:\n...", ...]`
- With `*` (original code): 
  - `seeds` = `"seeds: 79 14 55 13"`
  - `mappings` = `["seed-to-soil map:\n50 98 2\n52 50 48", "soil-to-fertilizer map:\n...", ...]`
- Without `*` (modified code):
  - `seeds` = `"seeds: 79 14 55 13"`
  - `mappings` = `"seed-to-soil map:\n50 98 2\n52 50 48"`
  - Plus, a `ValueError` if there are more than two elements after splitting.

In summary, without the `*`, your code would not work as intended unless the file content is structured to have exactly two sections. With the data you've provided, it would lead to an error because there are more than two sections in the file.

Certainly! Unpacking in Python is a powerful and flexible feature that allows for more readable and expressive ways to assign values to variables. It can be used with any iterable (like lists, tuples, strings, etc.), and there are several interesting ways it can be utilized:

### Basic Unpacking
The most basic form of unpacking assigns elements of an iterable to variables:

```python
a, b, c = [1, 2, 3]
```

Here, `a` becomes `1`, `b` becomes `2`, and `c` becomes `3`.

### Unpacking with `*`
The `*` operator can be used to grab excess items:

```python
first, *middle, last = [1, 2, 3, 4, 5]
```

In this case, `first` becomes `1`, `middle` becomes `[2, 3, 4]`, and `last` becomes `5`.

### Unpacking for Swapping Variables
Unpacking makes it easy to swap the values of two variables without needing a temporary variable:

```python
a, b = b, a
```

### Unpacking in Loops
Unpacking can be used in for-loops to iterate through a list of tuples or lists:

```python
for x, y in [(1, 2), (3, 4), (5, 6)]:
    print(x, y)
```

### Unpacking with Ignoring Values
Sometimes, you might not be interested in all the values. Use `_` to ignore them:

```python
a, _, c = (1, 2, 3)  # _ will be 2, but we are ignoring it
```

### Extended Unpacking (Python 3.x)
In Python 3, extended unpacking allows for even more flexibility:

```python
first, *rest, last = "Hello World"
```

`first` will be `'H'`, `last` will be `'d'`, and `rest` will be a list of all the other characters.

### Unpacking in Function Arguments
Unpacking can also be used when calling functions. The `*` operator unpacks a list or tuple into positional arguments, while `**` unpacks a dictionary into keyword arguments:

```python
def func(a, b, c):
    print(a, b, c)

args = [1, 2, 3]
func(*args)  # Equivalent to func(1, 2, 3)

kwargs = {'a': 1, 'b': 2, 'c': 3}
func(**kwargs)  # Equivalent to func(a=1, b=2, c=3)
```

### Nested Unpacking
Python also supports nested unpacking, which can be used to unpack nested iterables:

```python
(a, b), c = [1, 2], 3
```

Here, `a` becomes `1`, `b` becomes `2`, and `c` becomes `3`.

Unpacking is a feature that, when used properly, can significantly enhance the readability and succinctness of Python code. It allows for elegant solutions to common programming patterns and is a hallmark of Python's emphasis on readability and expressiveness.